In [1]:
import hashlib
import os
import numpy as np
import datetime
import csv
import time
from PIL import Image, ImageDraw, ImageFont
import json
import warnings
import cProfile
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from joblib import Parallel, delayed

In [2]:

# Suppress deprecation warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

def save_game_data(unique_id, metadata, game_states):
    folder_path = f'game_states/{unique_id}/'
    os.makedirs(folder_path, exist_ok=True)

    # Save metadata
    with open(f"{folder_path}metadata.json", "w") as f:
        json.dump(metadata, f)

    # Save game state history
    with open(f"{folder_path}game_state.csv", "w", newline='') as f:
        writer = csv.DictWriter(f, fieldnames=game_states[0].keys())
        writer.writeheader()
        for state in game_states:
            writer.writerow(state)
       
def random_shot(guess_board, valid_targets, shot_args):
    shot_idx = np.random.randint(valid_targets.shape[0])
    return tuple(valid_targets[shot_idx])
 
def empty_board(board_size):
    return np.full((board_size, board_size), 'E', dtype=str)  

def combat_turn(guess_board, board, player_lives, turn, player, shot_function, shot_args=None):
    valid_targets = np.argwhere(guess_board == 'E') #Empty tiles
    
    if valid_targets.size == 0: # No targets left
        game_state = {
        'Turn': turn,
        'Player': player,
        'Shot_Row': None,
        'Shot_Col': None,
        'Result': 'Surrender',
        'Player_Lives': player_lives}  
        return guess_board, board, player_lives, game_state
    else: # Targets exist
        shot = shot_function(guess_board, valid_targets, shot_args)
        check_hit = board[shot] # check if there is something at the shot's coordinates
        
        if check_hit == 'F':  # Fleet present at Shot -> Hit
            guess_board[shot] = board[shot] = result = 'H'
            player_lives -= 1
        else: # Miss
            guess_board[shot] = result = 'M'

        game_state = {
            'Turn': turn,
            'Player': player,
            'Shot_Row': shot[0],
            'Shot_Col': shot[1],
            'Result': result,
            'Player_Lives': player_lives
        }
        
        return guess_board, board, player_lives, game_state

def simple_game(unique_id):    
    board_size = 10
    fleet_size = 5
    board1, board2 = empty_board(board_size), empty_board(board_size)
    rng = np.random.default_rng()
    tiles = board_size * board_size
    fleet_positions1 = rng.choice(tiles, size=fleet_size, replace=False)
    fleet_positions2 = rng.choice(tiles, size=fleet_size, replace=False)
    fleet1_actual = np.unravel_index(fleet_positions1, (board_size, board_size))
    fleet2_actual = np.unravel_index(fleet_positions2, (board_size, board_size))
  
    board1[fleet1_actual] = 'F'
    board2[fleet2_actual] = 'F'
    guess_board1, guess_board2 = empty_board(board_size), empty_board(board_size)
    player1_lives, player2_lives = fleet_size, fleet_size
    game_states = []
    
    turn, shots1, shots2, hits1, hits2 = 0, 0, 0, 0, 0
    while (player1_lives > 0 or player2_lives > 0):
        turn += 1
        guess_board1, board2, player2_lives, game_state1 = combat_turn(guess_board1, board2, player2_lives, turn, 1)
        shots1 += 1
        hits1 += game_state1['Result'] == 'H'
        game_states.append(game_state1)
        
        guess_board2, board1, player1_lives, game_state2 = combat_turn(guess_board2, board1, player1_lives, turn, 2)
        shots2 += 1
        hits2 += game_state2['Result'] == 'H'
        game_states.append(game_state2)
    
    winner = (1 if player1_lives > 0 else 2)
    accuracy1, accuracy2 = accuracy(hits1, shots1), accuracy(hits2, shots2)
    metadata = {
        'Total_Turns': turn,
        'Winner': winner,
        'Player1_Accuracy': accuracy1,
        'Player2_Accuracy': accuracy2
    }
    
    save_game_data(unique_id, metadata, game_states)
    
    return metadata, np.copy(board1), np.copy(board2)

def generate_unique_id():
    unique_str = str(np.random.random()) + datetime.datetime.now().strftime("%Y%m%d%H%M%S%f")
    unique_id = hashlib.md5(unique_str.encode()).hexdigest()
    return unique_id[:16]

def draw_coordinates(draw, x_offset, y_offset, cell_size, board_size):
    # Define a font size that fits within the cell size
    font_size = int(cell_size * 0.3)
    font = ImageFont.load_default(font_size)
    
    for row in range(board_size):
        for col in range(board_size):
            x = x_offset + col * cell_size
            y = y_offset + row * cell_size
            
            # Define the label for each cell: columns are numbers and rows are letters
            label = f"{chr(65 + row)}{col + 1}"
            
            # Calculate text size and position to center it in the cell
            text_w, text_h = draw.textsize(label, font=font)
            text_x = x + (cell_size - text_w) // 2
            text_y = y + (cell_size - text_h) // 2
            
            draw.text((text_x, text_y), label, font=font, fill="black")
            
def draw_board(draw, board, x_offset, y_offset, cell_size, board_size, font=None):
    for row in range(board_size):
        for col in range(board_size):
            x1, y1 = x_offset + col * cell_size, y_offset + row * cell_size
            x2, y2 = x1 + cell_size, y1 + cell_size
            color = "white"
            if board[row, col] == 'E':
                color = "white"  # Empty
            elif board[row, col] == 'M':
                color = "blue"  # Miss
            elif board[row, col] == 'H':
                color = "red"  # Hit
            elif board[row, col] == 'F':
                color = "green"  # Fleet
            elif board[row, col] == 'S':
                color = "black"  # Separator
            draw.rectangle([x1, y1, x2, y2], fill=color)
            
            # Add small coordinate text
            if font and board[row, col] != 'S':
                coord_text = f"{chr(65 + row)}{col + 1}"
                text_w, text_h = draw.textsize(coord_text, font=font)
                text_x = x1 + (cell_size - text_w) // 2
                text_y = y1 + (cell_size - text_h) // 2
                draw.text((text_x, text_y), coord_text, fill="black", font=font)
            
def render_game_state_to_image(game_state, initial_board1, initial_board2, board_size=10, cell_size=20):
    board_size += 1  # Adding extra row and column for the separator
    img_size = board_size * cell_size * 2  # accommodate both boards for each player
    img = Image.new("RGB", (img_size, img_size), "white")
    draw = ImageDraw.Draw(img)
    
    board1 = empty_board(board_size)
    board1[:-1, :-1] = initial_board1  # Copying initial_board1 to board1
    board2 = empty_board(board_size)
    board2[:-1, :-1] = initial_board2  # Copying initial_board2 to board2
    guess_board1 = empty_board(board_size)
    guess_board2 = empty_board(board_size)
    
    # Fill last row and column with separators (4)
    board1[-1, :] = 'S'
    board1[:, -1] = 'S'
    board2[-1, :] = 'S'
    board2[:, -1] = 'S'
    guess_board1[-1, :] = 'S'
    guess_board1[:, -1] = 'S'
    guess_board2[-1, :] = 'S'
    guess_board2[:, -1] = 'S'

    for state in game_state:
        player = int(state['Player'])
        row = int(state['Shot_Row'])
        col = int(state['Shot_Col'])
        result = state['Result']

        if player == 1:
            guess_board2[row, col] = result  
            if result == 'H':
                board2[row, col] = 'H'
        else:
            guess_board1[row, col] = result  
            if result == 'H':
                board1[row, col] = 'H'

     # Font setup
    font = ImageFont.load_default()

    draw_board(draw, board1, 0, 0, cell_size, board_size, font=font)
    draw_board(draw, guess_board1, board_size * cell_size, 0, cell_size, board_size, font=font)
    draw_board(draw, board2, 0, board_size * cell_size, cell_size, board_size, font=font)
    draw_board(draw, guess_board2, board_size * cell_size, board_size * cell_size, cell_size, board_size, font=font)

    return img

def save_game_states_as_gif(game_states, initial_board1, initial_board2, output_path):
    frames = []
    accumulated_states = []
    for i in range(0, len(game_states), 2):
        accumulated_states.extend(game_states[i:i+2])
        img = render_game_state_to_image(accumulated_states, initial_board1, initial_board2)
        frames.append(img)
    frames[0].save(output_path, save_all=True, append_images=frames[1:], loop=0, duration=250)

def read_game_state_from_csv(unique_id):
    folder_path = os.path.join("game_states", unique_id)
    game_state_path = os.path.join(folder_path, 'game_state.csv')
    game_state = []
    try:
        with open(game_state_path, 'r') as f:
            reader = csv.DictReader(f)
            for row in reader:
                game_state.append(row)
    except FileNotFoundError:
        print(f"File not found: {game_state_path}")
    return game_state

def run_and_visualize_game():
    unique_id = generate_unique_id()
    metadata, initial_board1, initial_board2 = simple_game(unique_id)
    
    game_state = read_game_state_from_csv(unique_id)
    if game_state:
        output_gif_path = os.path.join("game_states", unique_id, 'game_animation.gif')
        save_game_states_as_gif(game_state, initial_board1, initial_board2, output_gif_path)
    else:
        print("Game state could not be loaded, skipping visualization.")

def enumerate_valid_positions(board_size, ship_length, wrap):
    valid_positions = []
    for row in range(board_size):
        for col in range(board_size):
            if wrap or (row + ship_length - 1 < board_size):
                valid_positions.append((row, col, 'down'))
            if wrap or (col + ship_length - 1 < board_size):
                valid_positions.append((row, col, 'right'))
    return valid_positions

def place_ship(board, start_row, start_col, direction, ship_length, wrap):
    board_size = board.shape[0]
    if direction == 'down': # down
        for i in range(ship_length):
            row = (start_row + i) % board_size if wrap else start_row + i
            board[row, start_col] = 'F'
    else: # right
        for i in range(ship_length):
            col = (start_col + i) % board_size if wrap else start_col + i
            board[start_row, col] = 'F'

def place_real_ships(board, ship_lengths, wrap):
    rng = np.random.default_rng()
    board_size = board.shape[0]
    
    for ship_length in ship_lengths:
        valid_positions = enumerate_valid_positions(board_size, ship_length, wrap)
        rng.shuffle(valid_positions)  # Shuffle the list first
        for start_row, start_col, direction in valid_positions:
            if direction == 'down':
                slice_to_check = board[start_row: start_row + ship_length, start_col]
            else:
                slice_to_check = board[start_row, start_col: start_col + ship_length]
            
            if all(x == 'E' for x in slice_to_check):
                place_ship(board, start_row, start_col, direction, ship_length, wrap)
                break  # Break out of the loop if ship is successfully placed
            
    return board

def accuracy(hits, shots):
    if shots == 0:
        return 0
    else:
        return np.round(np.divide(hits, shots) * 100, 2)

def real_ships_simple_game(unique_id, shot_functions, ship_lengths=[5, 4, 3, 3, 2], wrap=False):
        
    board_size = 10
    board1, board2 = empty_board(board_size), empty_board(board_size)
        
    # Place real ships on the boards
    board1 = place_real_ships(board1, ship_lengths, wrap)
    board2 = place_real_ships(board2, ship_lengths, wrap)
    print('Fleets placed')
    
    guess_board1, guess_board2 = np.full((board_size, board_size), 'E', dtype=str), np.full((board_size, board_size), 'Empty', dtype=str)
    # Lives are equal to the number of 'Fleet' tiles on the board, where (board == 'Fleet')
    player1_lives, player2_lives = np.sum(board1 == 'F'), np.sum(board2 == 'F')

    game_states = []
    
    turn, shots1, shots2, hits1, hits2 = 0, 0, 0, 0, 0
    while (player1_lives > 0 or player2_lives > 0):
        turn += 1
        if turn>250:
            # print("Turn limit reached, stopping game.")
            break
        guess_board1, board2, player2_lives, game_state1 = combat_turn(guess_board1, board2, player2_lives, turn, 1, shot_function=shot_functions[0])
        shots1 += 1
        hits1 += game_state1['Result'] == 'H'
        game_states.append(game_state1)
        
        guess_board2, board1, player1_lives, game_state2 = combat_turn(guess_board2, board1, player1_lives, turn, 2, shot_function=shot_functions[1])
        shots2 += 1
        hits2 += game_state2['Result'] == 'H'
        game_states.append(game_state2)
    
    winner = 1 if player1_lives > 0 else 2
    
    accuracy1, accuracy2 = accuracy(hits1, shots1), accuracy(hits2, shots2)
    metadata = {
        'Total_Turns': turn,
        'Winner': winner,
        'Player1_Accuracy': accuracy1,
        'Player1_Strategy': str(shot_functions[0]),
        'Player1_Setup': 'Random',
        'Player2_Accuracy': accuracy2,
        'Player2_Strategy': str(shot_functions[1]),
        'Player2_Setup': 'Random'
    }
    
    save_game_data(unique_id, metadata, game_states)
    return metadata, np.copy(board1), np.copy(board2)

def generate_one_game(shot_functions=None):
    unique_id = generate_unique_id()
    
    if shot_functions is None:
        shot_functions = [random_shot, random_shot]
    metadata, initial_board1, initial_board2 = real_ships_simple_game(unique_id, shot_functions)
    print("Metadata:", metadata)

    # Now visualize it
    game_state = read_game_state_from_csv(unique_id)
    if game_state:
        output_gif_path = os.path.join("game_states", unique_id, 'game_animation.gif')
        save_game_states_as_gif(game_state, initial_board1, initial_board2, output_gif_path)
    else:
        print("Game state could not be loaded, skipping visualization.")
    return unique_id

def generate_n_games(shot_functions=None, n=1, workers=1):
    game_ids = Parallel(n_jobs=workers)(delayed(generate_one_game)(shot_functions) for _ in range(n))
    return game_ids

def profile_generate_n_games():
    
    profiler = cProfile.Profile()
    profiler.enable()
    generate_n_games(99, workers=-1)
    profiler.disable()
    profiler.print_stats(sort='time')

def heuristic_shot(guess_board, valid_targets, shot_args=None):
    # look for cells adjacent to 'H' (hit)
    candidates = []
    for row, col in valid_targets:
        neighbors = [(row-1, col), (row+1, col), (row, col-1), (row, col+1)]
        if any(guess_board[r, c] == 'H' for r, c in neighbors if 0 <= r < guess_board.shape[0] and 0 <= c < guess_board.shape[1]):
            candidates.append((row, col))
    
    if candidates:
        return candidates[np.random.randint(len(candidates))]
    
    # fallback to random
    return random_shot(guess_board, valid_targets, shot_args)

def read_game_metadata(base_folder, strategy_name):
    metadata_list = []
    strategy_folder = os.path.join(base_folder, strategy_name)
    
    for game_folder in os.listdir(strategy_folder):
        full_path = os.path.join(strategy_folder, game_folder, "metadata.json")
        
        try:
            with open(full_path, 'r') as f:
                metadata = json.load(f)
            metadata_list.append(metadata)
        except FileNotFoundError:
            pass

    return pd.DataFrame(metadata_list)

def analyze_games(base_folder="game_states", strategy_name="random_shot"):
    metadata_df = read_game_metadata(base_folder, strategy_name)
    
    # Compute average accuracy for the strategy
    avg_accuracy_p1 = metadata_df['Player1_Accuracy'].mean()
    avg_accuracy_p2 = metadata_df['Player2_Accuracy'].mean()
    
    summary_stats = {
        "Player1_Avg_Accuracy": avg_accuracy_p1,
        "Player2_Avg_Accuracy": avg_accuracy_p2
    }
    
    # Save summary statistics to a JSON file in the strategy's subfolder
    summary_path = os.path.join(base_folder, strategy_name, "summary_stats.json")
    with open(summary_path, 'w') as f:
        json.dump(summary_stats, f)
        
    # Analyze and plot Density of Hits (Need game state data for this)
    hit_density = np.zeros((10, 10))

    for game_folder in os.listdir(os.path.join(base_folder, strategy_name)):
        full_path = os.path.join(base_folder, strategy_name, game_folder, "game_state.csv")
        
        try:
            game_state_df = pd.read_csv(full_path)
            for idx, row in game_state_df.iterrows():
                if row['Result'] == 'H':
                    hit_density[int(row['Shot_Row']), int(row['Shot_Col'])] += 1
        except FileNotFoundError:
            pass

    plt.figure()
    sns.heatmap(hit_density, cmap="YlGnBu", annot=True)
    plt.title("Density of Hits")
    
    # Save the heatmap to the strategy's subfolder
    heatmap_path = os.path.join(base_folder, strategy_name, "hit_density.png")
    plt.savefig(heatmap_path)
    plt.show()
shot_functions = [random_shot, heuristic_shot]
#generate_n_games(shot_functions, n=1, workers=1)

In [6]:
# ML Shot function
from sklearn.linear_model import LinearRegression
import numpy as np

# Initialize model
model = LinearRegression()

def trainable_shot(guess_board, valid_targets, shot_args=(model)):
    # look for cells adjacent to 'H' (hit)
    candidates = []
    for row, col in valid_targets:
        neighbors = [(row-1, col), (row+1, col), (row, col-1), (row, col+1)]
        if any(guess_board[r, c] == 'H' for r, c in neighbors if 0 <= r < guess_board.shape[0] and 0 <= c < guess_board.shape[1]):
            candidates.append((row, col))
    
    if candidates:
        return candidates[np.random.randint(len(candidates))]
    
    # fallback to random
    return random_shot(guess_board, valid_targets, shot_args)

shot_functions = [random_shot, trainable_shot]

generate_one_game(shot_functions)

Fleets placed


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()